<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-предобученной-модели-BERT" data-toc-modified-id="Загрузка-предобученной-модели-BERT-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка предобученной модели BERT</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка признаков</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression()" data-toc-modified-id="LogisticRegression()-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression()</a></span></li><li><span><a href="#LinearSVC()" data-toc-modified-id="LinearSVC()-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LinearSVC()</a></span></li><li><span><a href="#KNeighborsClassifier" data-toc-modified-id="KNeighborsClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>KNeighborsClassifier</a></span></li><li><span><a href="#DummyClassifier()" data-toc-modified-id="DummyClassifier()-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>DummyClassifier()</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.


# Цели исследования

Построить модель для классификации комментариев на позитивные и негативные со значением метрики качества F1 ***не меньше 0.75***.

# Описание данных

Данные находятся в файле `toxic_comments.csv`.


Столбец ***text*** в нём содержит текст комментария, а ***toxic*** — целевой признак.

# План работы

1. Загрузить и подготовить данные.
2. Обучить разные модели.
3. Выводы.

## Подготовка

**Импортируем необходимые библиотеки вначале документа (pandas, numpy, tqdm и другие).**

In [1]:
!pip install -q transformers

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import notebook

#nlp
import torch
import transformers
import nltk
import re

#ml
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

#models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier

**Считаем данные из csv-файла и сохраним в переменную `data`. Путь к файлу: `/datasets/toxic_comments.csv`**

In [3]:
pth1 = '/datasets/toxic_comments.csv'
pth2 = 'C:/Users/User/Desktop/DS Python/Projects/toxic_comments.csv'
pth3 = '/toxic_comments.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1, index_col=[0])
elif os.path.exists(pth2):
    data = pd.read_csv(pth2, index_col=[0])
elif os.path.exists(pth3):
    data = pd.read_csv('/toxic_comments.csv', index_col=[0])
else:
    print('Something is wrong')

**Выведем первые 5 строк датафрейма `data` на экран.**

In [4]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


**Выведем основную информацию о данном датафрейме с помощью метода info().**

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Оценим как между собой соотносятся классы (1 - токсичный комментарий, 0 - нетоксичный)

In [6]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [7]:
pd.DataFrame(round((data['toxic'].value_counts()/len(data['toxic']))*100, 2)).style.background_gradient('coolwarm')

,toxic
0,89.840000
1,10.160000


Количество токсичных комментариев 10% от общего числа комментариев. Видно, что есть дисбаланс классов, с которым в последующем нужно будет бороться. Иначе это скажется на значении метрики - мы получим не достоверные результаты.

Чтобы ускорить работус данными и сэкономить оперативную память, используем для работы часть данных, а именно 1000 строк.

In [8]:
data_sample = data.sample(1000, random_state=123).reset_index(drop=True)
pd.DataFrame(round((data_sample['toxic'].value_counts()/len(data_sample['toxic']))*100, 2)).style.background_gradient('coolwarm')

,toxic
0,90.300000
1,9.700000


In [9]:
data_sample['toxic'].value_counts()

0    903
1     97
Name: toxic, dtype: int64

Соотношение положительных и отрицательных комментариев сохранилось.

### Загрузка предобученной модели BERT

Используем для выполнения данного задания BERT.

(DistilBert - уменьшенная версия BERT, которая быстрее и легче BERT, но при этом сравнима в результативаности, хоть и немного уступает. Используя DistilBert не удалось достичь метрики f1 выше 0,7.)

In [10]:
model_class, tokenizer_class, pretrained_weights = (transformers.BertModel,
                                                    transformers.BertTokenizer,
                                                    'unitary/toxic-bert')

# Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, 
#                             transformers.DistilBertTokenizerFast, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Чтобы применить нейронку BERT, нужно привести данные к требуемому формату.

Токенизируем тексты - разобьем их на слова и слоги в удобном для BERT формате. Так как BERT может принимать только до 512 токенов, то ограничим длину токинизированных текстов.

In [11]:
tokenized = data_sample['text'].apply((lambda x: tokenizer.encode(x, max_length=512,
                                                                  truncation=True,
                                                                  add_special_tokens=True)))

После токенизации мы получаем тексты в виде списка токенов, а весь датасет - список списков.

Преобразуем наши данные в двумерный массив, чтобы обрабатывать тексты одновременно. Найдем список с максимальным количеством токенов, примем его длину за базовую, а в остальных списках дозаполним недостающие значение нулями.

In [12]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Размеры нашего нового двумерного дополненного массива:

In [13]:
padded.shape

(1000, 512)

Создадим маску. Чтобы BERT правильно распознал передаваемые ей значения, поясним ей, что нужно игнорировать нули.

In [14]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 512)

### Подготовка признаков

Функция model() запускает наши предложения через BERT. Начнем проебразование текстов в эмбеддинги с помощью библиотеки tqdm (нужна, чтобы наглядно показать индикатор прогресса).

Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча небольшим.

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

In [16]:
batch_size = 200

# сделаем пустой список для хранения эмбеддингов твитов
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    # преобразуем данные
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    # преобразуем маску
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

# ускорим вычисления функцией no_grad()
    with torch.no_grad():
        batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))

# из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/5 [00:00<?, ?it/s]

Соберём все эмбеддинги в матрицу признаков:

In [17]:
features = np.concatenate(embeddings)
target = data_sample['toxic']

Разделим признаки на обучающую и тестовую выборку (оставим на тестирование 15% от общей выборки).

In [18]:
train_features, test_features, train_target, test_target = train_test_split(features,
                                                                            target,
                                                                            test_size=0.15,
                                                                            stratify=target,
                                                                            random_state=0)
print('Размер обучающей выборки:', train_features.shape)
print('Размер тестовой выборки:', test_features.shape)

Размер обучающей выборки: (850, 768)
Размер тестовой выборки: (150, 768)


In [19]:
print(f"Количество строк в train_target по классам: {np.bincount(train_target)}")
print(f"Количество строк в test_target по классам: {np.bincount(test_target)}")

Количество строк в train_target по классам: [768  82]
Количество строк в test_target по классам: [135  15]


**Вывод:**
1. Импортировали необходимые библиотеки.
2. Считали данные из csv-файла.
3. Вывели общую информацию по преобразованному датасету:
    - в датасете 2 столбца и 159.292 строк;
    - общая занимаемая память 3.6+ MB.
4. Загрузили предобученную модель BERT.
5. Токенизировали тексты и создали эмбеддинги.
6. Подготовили признаки для обучения моделей.

## Обучение

**Рассмотрим несколько моделей (LogisticRegression, LinearSVC, KNeighborsClassifier).**

### LogisticRegression()

Построим модель LogisticRegression и посчитаем значение метрики f1 на тестовой и обучающей выборке.

In [20]:
logreg_param = {'C': range(1,3),
                'penalty': ['l1', 'l2'],
                'solver' : ['liblinear'],
                'max_iter': [2000]}

rsc_logreg = GridSearchCV(LogisticRegression(),
                         logreg_param,
                         scoring='f1',
                         cv=5,
                         n_jobs=-1,
                         verbose=2)
rsc_logreg = rsc_logreg.fit(train_features, train_target)

print('Лучшие параметры для модели LogisticRegression:', rsc_logreg.best_params_)
print("f1_train с лучшими параметрами:", abs(rsc_logreg.best_score_))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Лучшие параметры для модели LogisticRegression: {'C': 1, 'max_iter': 2000, 'penalty': 'l1', 'solver': 'liblinear'}
f1_train с лучшими параметрами: 0.9009490061909418


### LinearSVC()

Построим модель LinearSVC и посчитаем значение метрики f1 на тестовой и обучающей выборке.

In [21]:
SVC_parameters = {'C': range(1, 20),
                 'max_iter': [2000]}

rsc_SVC = RandomizedSearchCV(LinearSVC(), SVC_parameters,
                  cv=5,
                  scoring='f1',
                  random_state=0,
                  n_jobs=-1,
                  verbose=1)
rsc_SVC = rsc_SVC.fit(train_features, train_target)

print('Лучшие параметры для модели LinearSVC:', rsc_SVC.best_params_)
print("f1_train с лучшими параметрами:", abs(rsc_SVC.best_score_))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Лучшие параметры для модели LinearSVC: {'max_iter': 2000, 'C': 2}
f1_train с лучшими параметрами: 0.8625963621030035


### KNeighborsClassifier

Построим модель KNeighborsClassifier и посчитаем значение метрики f1 на тестовой и обучающей выборке.

In [22]:
knn_parameters = {'n_neighbors': range(2, 15)}

rsc_knn = GridSearchCV(KNeighborsClassifier(), knn_parameters,
                  cv=3,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
rsc_knn = rsc_knn.fit(train_features, train_target)

print('Лучшие параметры для модели LinearSVC:', rsc_knn.best_params_)
print("f1_train с лучшими параметрами:", abs(rsc_knn.best_score_))

Fitting 3 folds for each of 13 candidates, totalling 39 fits
Лучшие параметры для модели LinearSVC: {'n_neighbors': 7}
f1_train с лучшими параметрами: 0.9228685320639344


### DummyClassifier()

Построим случайную модель DummyClassifier и посчитаем значение метрики f1 на тестовой выборке.

In [23]:
model = DummyClassifier(strategy = 'stratified')

model.fit(train_features, train_target)
pred = model.predict(test_features)
f1_dummy = f1_score(test_target, pred)
print('f1 Dummy: ', round(f1_dummy, 3))

f1 Dummy:  0.138


Запишем все параметры, которые мы получили в таблицу.

In [24]:
data = {'f1_train':[rsc_logreg.best_score_, rsc_SVC.best_score_, rsc_knn.best_score_, 'NaN'],
        'f1_test':['NaN', 'NaN', 'NaN', f1_dummy]}

table = pd.DataFrame(data)
table.index = ['LogisticRegression', 'LinearSVC', 'KNeighborsClassifier', 'Dummy']

pd.set_option('display.float_format', '{:.2f}'.format)
display(table)

,f1_train,f1_test
LogisticRegression,0.90,NaN
LinearSVC,0.86,NaN
KNeighborsClassifier,0.92,NaN
Dummy,NaN,0.14


Из таблицы видно, что самая лучшая метрика получается у модели KNeighborsClassifier. Рассчитаем метрику на тестовой выборке.

In [25]:
model_knn = KNeighborsClassifier(n_neighbors=7)
model_knn.fit(train_features, train_target)
pred_knn = model_knn.predict(test_features)
f1_knn = f1_score(test_target, pred_knn)
print('f1 KNeighborsClassifie тестовое значение: ', round(f1_knn, 2))

f1 KNeighborsClassifie тестовое значение:  0.97


## Выводы

В связи с тем, что вычисления объемные - удалось подобрать модель (BERT, также была опробована AutoModelForSequenceClassification и AutoModelForMaskedLM, которые на данном то этапе заполняли всю оперативную память и расчет прекращался), количество сэмплов и батчей для обучения модели и дальнейшего предсказания.

По данным предсказаниям токсичных комментариев лучшая метрика f1 получилась на модели `KNeighborsClassifier`. На тестовых данных метрика достигла результата 0,97.

Стоит отметить, что модели `LinearSVC` и `LogisticRegression` показали результат выше требуемого (0,75).